In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
import torch
import torch.nn.functional as F

# koelectra

In [2]:
def infer():
    text = "지나가는 사람을 때려서 전치 3주를 입혔어요. 처벌 받을까요?"
    base_model_name = "monologg/koelectra-small-v3-discriminator"
    model = AutoModelForSequenceClassification.from_pretrained(
        "kfkas/legal-question-filter-koelectra",
        num_labels=2,
        ignore_mismatched_sizes=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits.detach().cpu()
    pr = F.softmax(logits).numpy()
    arg = np.argmax(pr, axis=1)
    print(logits)
    print(pr)
    print(int(arg))
    if int(arg) == 0 and (pr[0][0] >= 0.98).all():
        print("법률입니다")
    else:
        print("법률 질문이 아닙니다(bard,chatgpt 등등 다른 API 호출하면 좋을거 같아용)")

In [3]:
if __name__ == "__main__":
    infer()

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/56.6M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

tensor([[ 5.3626, -4.9673]])
[[9.9996734e-01 3.2642944e-05]]
0
법률입니다


/tmp/ipykernel_24/2840614190.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pr = F.softmax(logits).numpy()
/tmp/ipykernel_24/2840614190.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(int(arg))
/tmp/ipykernel_24/2840614190.py:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if int(arg) == 0 and (pr[0][0] >= 0.98).all():


# 라이브러리

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import  AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import shutil

2024-07-23 09:29:42.615901: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 09:29:42.616008: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 09:29:42.767037: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# 데이터 불러오기
data_path = '/kaggle/input/qa-data-2/QA_data.csv'
df = pd.read_csv(data_path)

In [6]:
# 'text' 열에 텍스트 데이터가 있고 'label' 열에 레이블이 있다고 가정합니다.
df = df[['question', 'label']]

In [7]:
# 학습 데이터와 검증 데이터로 분할
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
# pandas DataFrame을 Hugging Face Datasets로 변환
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [9]:
# 토크나이저와 모델 불러오기
model_name = "monologg/koelectra-small-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

pytorch_model.bin:   0%|          | 0.00/56.6M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 데이터셋 토큰화
def preprocess_function(examples):
    return tokenizer(examples['question'], truncation=True)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/102068 [00:00<?, ? examples/s]

Map:   0%|          | 0/25517 [00:00<?, ? examples/s]

In [11]:
# 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
# 평가 지표 계산 함수 정의
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    return {
        "accuracy": accuracy,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [13]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=10,
    report_to='none'    
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [15]:
# 모델 학습
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.022600,0.016716,0.995101,0.995102,0.995107,0.995101
2,0.014100,0.020813,0.994513,0.994511,0.994538,0.994513
3,0.010200,0.013790,0.996473,0.996474,0.996479,0.996473
4,0.005800,0.014490,0.996826,0.996826,0.996826,0.996826
5,0.004200,0.020957,0.996159,0.996159,0.996164,0.996159
6,0.002100,0.019718,0.996512,0.996511,0.996516,0.996512


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=19140, training_loss=0.015599019705564127, metrics={'train_runtime': 6775.3533, 'train_samples_per_second': 90.388, 'train_steps_per_second': 2.825, 'total_flos': 1.7885609965805184e+16, 'train_loss': 0.015599019705564127, 'epoch': 6.0})

In [16]:
# 모델 평가
eval_results = trainer.evaluate()
print("Evaluation results:")
print(eval_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluation results:
{'eval_loss': 0.0197183508425951, 'eval_accuracy': 0.9965121291687894, 'eval_f1': 0.9965114830039471, 'eval_precision': 0.9965160512429897, 'eval_recall': 0.9965121291687894, 'eval_runtime': 107.6056, 'eval_samples_per_second': 237.135, 'eval_steps_per_second': 7.416, 'epoch': 6.0}


In [17]:
# 모델 저장
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")

('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json',
 './results/tokenizer.json')

# 학습된 모델 불러오기

In [18]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch

In [19]:
load_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/bert-kor-base-lawqa/bert-kor-base-lawQA')
load_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/bert-kor-base-lawqa/bert-kor-base-lawQA')

OSError: Incorrect path_or_model_id: '/kaggle/input/bert-kor-base-lawqa/bert-kor-base-lawQA'. Please provide either the path to a local folder or the repo_id of a model on the Hub.